In [2]:
import pickle
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')

import requests
from sentence_transformers import SentenceTransformer, util

with open('/content/drive/MyDrive/DVA/X_scaler_full_v1.pkl', 'rb') as file:
    scaler = pickle.load(file)

with open('/content/drive/MyDrive/DVA/xgboost_embedding_Lower_full_v1.pkl', 'rb') as file:
    best_model = pickle.load(file)

import torch.nn.functional as F
from sentence_transformers import SentenceTransformer
revision = None  # Replace with the specific revision to ensure reproducibility if the model is updated.
model = SentenceTransformer("avsolatorio/GIST-small-Embedding-v0", revision=revision)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [03:57:59] WARNING: /workspace/src/collective/../data/../common/error_msg.h:80: If you are loading a serialized model (like pickle in Python, RDS in R) or
configuration generated by an older version 

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/68.0k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import numpy as np
import pandas as pd
from xgboost import DMatrix, train

In [6]:
bdpt_dict={}
bdpt_dict[30]='SHOULDER'
bdpt_dict[31]='UPPERTRUNK'
bdpt_dict[32]='ELBOW'
bdpt_dict[33]='LOWERARM'
bdpt_dict[34]='WRIST'
bdpt_dict[35]='KNEE'
bdpt_dict[36]='LOWERLEG'
bdpt_dict[37]='ANKLE'
bdpt_dict[38]='PUBICREGION'
bdpt_dict[75]='HEAD'
bdpt_dict[76]='FACE'
bdpt_dict[77]='EYEBALL'
bdpt_dict[79]='LOWERTRUNK'
bdpt_dict[80]='UPPERARM'
bdpt_dict[81]='UPPERLEG'
bdpt_dict[82]='HAND'
bdpt_dict[83]='FOOT'
bdpt_dict[88]='MOUTH'
bdpt_dict[89]='NECK'
bdpt_dict[92]='FINGER'
bdpt_dict[93]='TOE'
bdpt_dict[94]='EAR'

In [7]:
def create_df(X):

    sample_df_single_record=pd.DataFrame(X,columns=['Age', 'Sex','Location','Product_1' ,'activity_at_injury', 'object_involved','injury_mechanism','More_details'])

    sample_df_single_record["Age"] = pd.to_numeric(sample_df_single_record["Age"])
    sample_df_single_record["Sex"] = pd.to_numeric(sample_df_single_record["Sex"])
    sample_df_single_record["Location"] = pd.to_numeric(sample_df_single_record["Location"])
    sample_df_single_record["Product_1"] = pd.to_numeric(sample_df_single_record["Product_1"])

    sample_df_single_record['Narrative']=sample_df_single_record["activity_at_injury"].astype(str) + ' '+sample_df_single_record["injury_mechanism"].astype(str)+ ' ' + sample_df_single_record["object_involved"].astype(str)+' '+sample_df_single_record["More_details"].astype(str)

    Embedding_df=pd.DataFrame(model.encode(sample_df_single_record['Narrative'], convert_to_tensor=True).numpy())
    Embedding_df.columns=Embedding_df.columns.astype(str)

    sample_df_single_record_2=sample_df_single_record[["Age","Sex","Location","Product_1"]]

    return Embedding_df,sample_df_single_record_2

In [18]:
def gen_prediction(bdpt_dict,data_ready,scaler,best_model): #,sematic_distance_bert


    prob_out={}

    k=0

    for i in bdpt_dict:
        data_ready.at[0,'Body_Part']=i

        X_test_scaled = scaler.transform(data_ready.select_dtypes(include=['number']))
        X_test_final=pd.DataFrame(X_test_scaled, columns=data_ready.select_dtypes(include=['number']).columns)
        dtest = DMatrix(data=X_test_final)
        y_prob = best_model.predict(dtest)[0]
        #print(y_prob)
        prob_out[str(i)]=y_prob
        k+=1

    return prob_out

In [13]:
def Predict_Body_parts_Prob(Age, Sex,Location,Product_1 ,activity_at_injury, object_involved,injury_mechanism,More_details):

    X = np.column_stack([Age, Sex,Location,Product_1 ,activity_at_injury, object_involved,injury_mechanism,More_details])
    embedding_df,sample_df_single_record_2 = create_df(X)
    data = {"Body_Part": [25],}
    df_body= pd.DataFrame(data)
    data_ready = pd.concat([sample_df_single_record_2[['Age','Sex']],df_body,sample_df_single_record_2[['Location','Product_1']], embedding_df], axis=1)
    #print(data_ready)
    output = gen_prediction(bdpt_dict,data_ready,scaler,best_model)

    return output

In [19]:
Predict_Body_parts_Prob(25, 1,1,3258 ,"fall walk", "bike","","fall from bike when riding")

{'30': 0.0063282475,
 '31': 0.025913913,
 '32': 0.0060901833,
 '33': 0.01754937,
 '34': 0.008896929,
 '35': 0.008241552,
 '36': 0.026210131,
 '37': 0.013994965,
 '38': 0.07756247,
 '75': 0.021943996,
 '76': 0.011409418,
 '77': 0.035655342,
 '79': 0.017622773,
 '80': 0.030592808,
 '81': 0.11721332,
 '82': 0.0019448939,
 '83': 0.0030811923,
 '88': 0.0038050055,
 '89': 0.012099536,
 '92': 0.0039061438,
 '93': 0.003825981,
 '94': 0.0035866464}